# Projeto Final - Machine Learning

## Alunos:
- Guilherme Fontana Louro
- Pedro Altobelli Teixeira Pinto

## Objetivo
- Criar um modelo de Machine Learning capaz de sugerir uma boa aposta para uma partida da premier league, a liga mais valiosa e competitiva do mundo. Essa aposta pode ser o resultado final do jogo, número de gols na partida(over/under), ou até handicap asiático.

## Dataset
- O dataset utilizado foi o [English Premier League](https://www.kaggle.com/datasets/saife245/english-premier-league), disponível no Kaggle. Ele contém dados de jogos de futebol da premier league por 20 temporadas, desde a temporada 2000/2001 até a 2021/2022. Os dados foram coletados do site [football-data.co.uk](https://www.football-data.co.uk/).

## Primeira iteração:
- Imports necessários:

In [158]:
import pandas as pd
import pathlib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import re
import glob

- Vamos concatenar os datasets de cada temporada em um único dataset, para facilitar a análise e a criação do modelo de machine learning.

In [159]:
# Lista de caminhos para os arquivos CSV de cada temporada
paths = glob.glob('../data/seasons/*.csv')

# Lista para armazenar os datasets de cada temporada
datasets = []

# Carregar os datasets de cada temporada
for path in paths:
    # Extrair o nome da temporada do caminho do arquivo
    temporada = path.split('/')[-1].split('.')[0]
    
    # Carregar o dataset da temporada e adicionar a coluna 'Temporada'
    dataset = pd.read_csv(path)
    dataset['Temporada'] = temporada
    
    # Adicionar o dataset à lista
    datasets.append(dataset)

# Concatenar os datasets em um único dataframe
data = pd.concat(datasets, ignore_index=True)

# Substituir valores ausentes por valores nulos (NaN)
data.replace('?', pd.NA, inplace=True)

# Exibir os 5 ultimos registros
print(data.tail())

     Div        Date     HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  HTAG  \
8395  E0  28/05/2023      Everton  Bournemouth     1     0   H     0     0   
8396  E0  28/05/2023        Leeds    Tottenham     1     4   A     0     1   
8397  E0  28/05/2023    Leicester     West Ham     2     1   H     1     0   
8398  E0  28/05/2023   Man United       Fulham     2     1   H     1     1   
8399  E0  28/05/2023  Southampton    Liverpool     4     4   D     2     2   

     HTR  ...  AvgC<2.5  AHCh  B365CAHH  B365CAHA  PCAHH  PCAHA  MaxCAHH  \
8395   D  ...      2.14 -1.00      2.02      1.77   2.10   1.81     2.17   
8396   A  ...      2.50  0.25      1.84      2.06   1.83   2.10     1.90   
8397   H  ...      2.51 -0.25      1.75      2.05   1.85   2.06     1.90   
8398   D  ...      2.95 -1.25      1.98      1.92   1.98   1.93     2.07   
8399   D  ...      3.22  1.25      1.82      2.08   1.85   2.07     1.96   

      MaxCAHA  AvgCAHH  AvgCAHA  
8395     1.92     2.03     1.83  
8396  

Inicialmente, foi pensado no projeto a possibilidade de usar o dataset de usando como base os dados, de forma que iria ser treinado um modelos com as informações de todas as temporadas anteriores e testado com a temporada atual. Esse será o primeiro modelo a ser testado.

A primeira iteração se basearia apenas em tentar prever os resultados dos jogos. Para isso, devemos concatenar os datasets de cada temporada em um único dataset, para facilitar a análise e a criação do modelo de machine learning, separando os dados de treino e teste, sendo o teste a temporada atual. Isso pode ser observado no código abaixo:

In [160]:
# Filtrar os resultados antes de 31/05/2022
train_data = data[data['Date'] < '2022-05-31']

# Selecionar as colunas relevantes para treinamento
train_data = train_data[["HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "FTR"]]


# Converta as variáveis categóricas em variáveis numéricas usando one-hot encoding
train_data_encoded = pd.get_dummies(train_data, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separar as features (variáveis de entrada) e o target (variável de saída)
X_train = train_data_encoded.drop("FTR", axis=1)
y_train = train_data_encoded["FTR"]

# Filtrar os resultados após 31/05/2022
test_data = data[data['Date'] > '2022-05-31']

# Selecionar as colunas relevantes para teste
test_data = test_data[["HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "FTR"]]

# Converta as variáveis categóricas em variáveis numéricas usando one-hot encoding
test_data_encoded = pd.get_dummies(test_data, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separar as features (variáveis de entrada) e o target (variável de saída)
X_test = test_data_encoded.drop("FTR", axis=1)
y_test = test_data_encoded["FTR"]

Agora que já temos os dados separados em treino e teste, podemos começar a análise exploratória dos dados. Para isso, pensamos em uma série de modelos que poderiamos utilizar, sendo eles:
- Regressão Logística
- Decision Tree
- Random Forest
- SVM
- Naive Bayes

Abaixo, segue o código para a criação de cada um dos modelos:

In [161]:
# Crie e ajuste o modelo de regressão logística
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Crie e ajuste o modelo de árvore de decisão
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

# Crie e ajuste o modelo de Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Crie e ajuste o modelo SVM
svm = SVC()
svm.fit(X_train, y_train)

# Crie e ajuste o modelo Naive Bayes
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)

c:\Users\guifl\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GaussianNB()

Inicialmente, foi pensado em prever os resultados apenas utilizando as informações dos nomes dos times, mas a acurácia foi muito baixa, então decidimos utilizar as informações de médias de gols nesses jogos, chutes e chutes no gol de ambos os times, além do histórico de resultados desses times. Para isso, criamos duas funções, uma que calcula as estatísticas desse confronto, e outra que calcula essas médias e adiciona ao dataset, como pode ser visto abaixo:

In [162]:
def obter_estatisticas_confronto(home_team, away_team):
    # Filtrar os jogos anteriores entre os times
    confrontos = data[((data['HomeTeam'] == home_team) & (data['AwayTeam'] == away_team)) |
                      ((data['HomeTeam'] == away_team) & (data['AwayTeam'] == home_team))]

    # Calcular as estatísticas dos confrontos
    total_jogos = confrontos.shape[0]
    vitorias_home = confrontos[confrontos['HomeTeam'] == home_team]['FTR'].eq('H').sum()
    vitorias_away = confrontos[confrontos['AwayTeam'] == home_team]['FTR'].eq('A').sum()
    empates = confrontos[confrontos['FTR'] == 'D'].shape[0]

    # Criar DataFrame com as estatísticas
    estatisticas = pd.DataFrame({'TotalJogos': [total_jogos],
                                 'VitoriasHome': [vitorias_home],
                                 'VitoriasAway': [vitorias_away],
                                 'Empates': [empates]})

    return estatisticas


In [163]:
def obter_medias_times(home_team, away_team):
    # Filtrar os jogos do time
    jogos = data[(data['HomeTeam'] == home_team) & (data['AwayTeam'] == away_team)]

    # Calcular as médias de gols, chutes e chutes no alvo
    home_media_gols = jogos['FTHG'].mean()
    home_media_chutes = jogos['HS'].mean()
    home_media_chutes_alvo = jogos['HST'].mean()
    away_media_gols = jogos['FTAG'].mean()
    away_media_chutes = jogos['AS'].mean()
    away_media_chutes_alvo = jogos['AST'].mean()

    # Criar DataFrame com as médias
    medias = pd.DataFrame({'HomeTeam': [home_team],
                            'AwayTeam': [away_team],
                            'FTHG': [home_media_gols],
                            'HS': [home_media_chutes],
                            'HST': [home_media_chutes_alvo],
                            'FTAG': [away_media_gols],
                            'AS': [away_media_chutes],
                            'AST': [away_media_chutes_alvo]})

    # Adicionar colunas de codificação dos times
    for col in X_train.columns:
        if col.startswith("HomeTeam_") and col != f"HomeTeam_{home_team}":
            medias[col] = 0
        elif col.startswith("AwayTeam_") and col != f"AwayTeam_{away_team}":
            medias[col] = 0
        elif col.startswith("HomeTeam_") and col == f"HomeTeam_{home_team}":
            medias[col] = 1
        elif col.startswith("AwayTeam_") and col == f"AwayTeam_{away_team}":
            medias[col] = 1
    
    # Reordenar as colunas
    medias = medias[X_train.columns]
    

    # Adicionar colunas ausentes
    missing_cols = set(X_train.columns) - set(medias.columns)
    for col in missing_cols:
        medias[col] = 0

    return medias

Agora, ambas as funções ofertarão mais informações para o modelo, possibilitando a criação de uma função capaz de prever os resultados dos jogos. Essa função será a função de previsão, que recebe um jogo, com os nomes de ambos os times, e retorna o resultado da partida, como pode ser visto abaixo:

In [164]:
db = pathlib.Path.cwd() / "../data/seasons/2022-2023.csv"
lastseason = pd.read_csv(db)

# Função para fazer uma sugestão de aposta com base no modelo treinado
def aposta_resultado(jogo, modelo):
    jogo_encoded = obter_medias_times(jogo['HomeTeam'], jogo['AwayTeam'])

    # Fazer a previsão com base no modelo treinado
    predicao = modelo.predict(jogo_encoded)

    # Filtrar o jogo específico
    jogo_especifico = lastseason[(lastseason['HomeTeam'] == jogo['HomeTeam']) & (lastseason['AwayTeam'] == jogo['AwayTeam'])]

    # Obter as odds específicas do jogo
    odd_home = jogo_especifico['B365H'].values[0]
    odd_draw = jogo_especifico['B365D'].values[0]
    odd_away = jogo_especifico['B365A'].values[0]

    # Se a previsão for de vitória do time da casa
    if predicao[0] == 'H':
        sugestao = "Sugestão de aposta: Vitória do time da casa com odd de {}".format(odd_home)
    # Se a previsão for de empate
    elif predicao[0] == 'D':
        sugestao = "Sugestão de aposta: Empate com odd de {}".format(odd_draw)
    # Se a previsão for de vitória do time visitante
    else:
        sugestao = "Sugestão de aposta: Vitória do time visitante com odd de {}".format(odd_away)

    return sugestao, predicao[0]


Para testar a qualidade do modelo, podemos utilizar a função de previsão para prever os resultados dos jogos da temporada atual, e comparar com os resultados reais. Aqui, vamos gerar uma sugestão para o jogo onde o Man United recebe o Arsenal em casa, um grande clássico do futebol inglês. Além disso, vamos comparar a previsão com o resultado real da partida:

In [165]:
# Carregar o modelo treinado
modelo = decision_tree

# Exemplo de jogo específico
jogo = {
    "HomeTeam": 'Man United',
    "AwayTeam": 'Arsenal',
}

# Fazer a sugestão de aposta com base no modelo treinado
sugestao_aposta = aposta_resultado(jogo, modelo)

print(sugestao_aposta[0])

Sugestão de aposta: Vitória do time da casa com odd de 2.5


## Agora podemos comparar com o resultado na vida real:
![Resultado United x Arsenal](../imgs/unitedxarsenal.png)

Apesar disso, o modelo ainda não é perfeito, já que no futebol tudo pode acontecer, e o resultado de um jogo pode ser influenciado por diversos fatores, como lesões, suspensões, desfalques, etc. Um exemplo disso é o jogo entre o Manchester United e o Manchester City, onde o United venceu por 2x0, mesmo sendo o azarão da partida, como pode ser visto abaixo:

In [179]:
# Carregar o modelo treinado
modelo = decision_tree

# Exemplo de jogo específico
jogo = {
    "HomeTeam": 'Man United',
    "AwayTeam": 'Man City',
}

# Fazer a sugestão de aposta com base no modelo treinado
sugestao_aposta = aposta_resultado(jogo, modelo)

print(sugestao_aposta[0])

Sugestão de aposta: Vitória do time visitante com odd de 1.85


## Agora podemos comparar com o resultado na vida real:
![Resultado United x City](../imgs/unitedxcity.png)

Dessa forma, é interessante medir a acurácia do modelo, para saber se ele é realmente bom. Para isso, vamos criar uma função que recebe os resultados reais e os resultados previstos, e retorna a acurácia do modelo, como pode ser visto abaixo:

In [167]:
def calcular_acuracia(modelo):
    # Criar uma lista com os 20 times da Premier League 2021/2022
    times = lastseason['HomeTeam'].unique()

    # Criar uma acuracia total para todos os times
    acuracia_total = 0

    # Criar um for que percorre todos os times e faz a sugestão de aposta
    for time in times:
        # Criar um dataframe com os jogos do time
        jogos_time = lastseason[(lastseason['HomeTeam'] == time) | (lastseason['AwayTeam'] == time)].copy()

        # Criar um for que percorre todos os jogos do time e faz a sugestão de aposta
        for index, jogo in jogos_time.iterrows():
            # Criar um dicionário com os dados do jogo
            jogo = {
                "HomeTeam": jogo['HomeTeam'],
                "AwayTeam": jogo['AwayTeam'],
            }

            # Fazer a sugestão de aposta
            sugestao_aposta = aposta_resultado(jogo, modelo)

            # Criar uma coluna com a sugestão de aposta
            jogos_time.loc[index, 'SugestaoAposta'] = sugestao_aposta[1]

        # Criar uma coluna com o resultado da aposta
        jogos_time.loc[:, 'ResultadoAposta'] = jogos_time['FTR'].eq(jogos_time['SugestaoAposta'].str[-1])

        # Calcular a acurácia do modelo
        acuracia = jogos_time['ResultadoAposta'].mean()

        # Somar a acurácia do time na acurácia total
        acuracia_total += acuracia

    # Retornar a acurácia do modelo
    return acuracia_total / len(times)


Vamos testar a acurácia do Random Forest, que é uma evolução do Decision Tree, e o SVM, que é um modelo que utiliza vetores para classificar os dados. Para isso, vamos criar uma função que recebe o modelo, os dados de treino e os dados de teste, e retorna a acurácia do modelo, como pode ser visto abaixo:

In [168]:
print("A acurácia do modelo de regressão logística é de {}%".format(round(calcular_acuracia(logreg) * 100, 2)))
print("A acurácia do modelo de decision tree é de {}%".format(round(calcular_acuracia(decision_tree) * 100, 2)))
print("A acurácia do modelo de random forest é de {}%".format(round(calcular_acuracia(random_forest) * 100, 2)))
print("A acurácia do modelo support vector machine é de {}%".format(round(calcular_acuracia(svm) * 100, 2)))
print("A acurácia do modelo naive bayes é de {}%".format(round(calcular_acuracia(naive_bayes) * 100, 2)))

A acurácia do modelo de regressão logística é de 61.84%
A acurácia do modelo de decision tree é de 62.11%
A acurácia do modelo de random forest é de 62.37%
A acurácia do modelo support vector machine é de 61.32%
A acurácia do modelo naive bayes é de 47.11%


# Iteração 2: Over ou Under 2.5 gols
Como os resultados na última iteração foram bons, mas ainda estão muito longe de serem perfeitos, vamos criar outra iteração, capaz de sugerir uma aposta para a quantidade de gols que vão sair em uma partida. Nessa segunda iteração, vamos precisar treinar um novo modelo que recebe as informações da partida, mas ao invés de prever o resultado, ele vai prever a quantidade de gols que vão sair na partida. Para isso, vamos criar uma função que recebe os dados de treino e teste, como pode ser visto abaixo:

In [169]:
# Filtrar os resultados antes de 31/05/2022
gols_train = data[data['Date'] < '2022-05-31']

# Selecionar as colunas relevantes para treinamento
gols_train = gols_train[["HomeTeam", "AwayTeam", "FTHG", "FTAG"]]

# Transformar a variável de saída em uma variável binária indicando se teve mais ou menos de 2.5 gols
gols_train['Resultado'] = gols_train['FTHG'] + gols_train['FTAG'] > 2.5
gols_train['Resultado'] = gols_train['Resultado'].astype(int)

# Converter as variáveis categóricas em variáveis numéricas usando one-hot encoding
gols_train_encoded = pd.get_dummies(gols_train, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separar as features (variáveis de entrada) e o target (variável de saída)
X_train_gols = gols_train_encoded.drop("Resultado", axis=1)
y_train_gols = gols_train_encoded["Resultado"]

# Filtrar os resultados a partir de 31/05/2022
test_gols = data[data['Date'] >= '2022-05-31']

# Selecionar as colunas relevantes para teste
test_gols = test_gols[["HomeTeam", "AwayTeam", "FTHG", "FTAG"]]
test_gols['Resultado'] = test_gols['FTHG'] + test_gols['FTAG'] > 2.5
test_gols['Resultado'] = test_gols['Resultado'].astype(int)

# Converter as variáveis categóricas em variáveis numéricas usando one-hot encoding
test_gols_encoded = pd.get_dummies(test_gols, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separar as features (variáveis de entrada) e o target (variável de saída)
X_test_gols = test_gols_encoded.drop("Resultado", axis=1)
y_test_gols = test_gols_encoded["Resultado"]

Agora, vamos treinar os mesmos modelos com base nesse novo dataset de treino:

In [170]:
# Crie e ajuste o modelo de regressão logística
logreg_gols = LogisticRegression()
logreg_gols.fit(X_train_gols, y_train_gols)

# Crie e ajuste o modelo de árvore de decisão
decision_tree_gols = DecisionTreeClassifier(random_state=42)
decision_tree_gols.fit(X_train_gols, y_train_gols)

# Crie e ajuste o modelo de Random Forest
random_forest_gols = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_gols.fit(X_train_gols, y_train_gols)

# Crie e ajuste o modelo SVM_gols
svm_gols = SVC()
svm_gols.fit(X_train_gols, y_train_gols)

# Crie e ajuste o modelo Naive Bayes
naive_bayes_gols = GaussianNB()
naive_bayes_gols.fit(X_train_gols, y_train_gols)

GaussianNB()

Agora, vamos criar uma função que recebe um jogo e um modelo, retornando a quantidade de gols que vão sair na partida(over ou under 2.5), como pode ser visto abaixo:

In [171]:
# Função para fazer uma sugestão de aposta com base no modelo treinado
def aposta_gols(jogo, modelo):
    jogo_encoded = obter_medias_times(jogo['HomeTeam'], jogo['AwayTeam'])
    # cortar as colunas AS, AST, HS e HST
    jogo_encoded = jogo_encoded.drop(['AS', 'AST', 'HS', 'HST'], axis=1)


    # Fazer a previsão com base no modelo treinado
    predicao = modelo.predict(jogo_encoded)

    # Filtrar o jogo específico
    jogo_especifico = lastseason[(lastseason['HomeTeam'] == jogo['HomeTeam']) & (lastseason['AwayTeam'] == jogo['AwayTeam'])]

    # Obter as odds específicas do jogo
    odd_maior = jogo_especifico['B365>2.5'].values[0]
    odd_menor = jogo_especifico['B365<2.5'].values[0]

    # Criar o dicionário com a sugestão de aposta
    sugestao_aposta = {
        'HomeTeam': jogo['HomeTeam'],
        'AwayTeam': jogo['AwayTeam'],
        'Predicao': predicao[0],
        'OddMaior': odd_maior,
        'OddMenor': odd_menor,
    }

    # Verificar se o jogo teve mais de 2.5 gols ou não
    if predicao[0]:
        sugestao = "Sugestão de aposta: Mais de 2.5 gols com odd de {}".format(odd_maior)
    else:
        sugestao = "Sugestão de aposta: Menos de 2.5 gols com odd de {}".format(odd_menor)

    return sugestao, predicao[0]

Agora, vamos testar o modelo com o jogo entre o Manchester United e o Arsenal, dessa vez o jogo fora de casa, como pode ser visto abaixo:

In [172]:
# Carregar o modelo treinado
modelo = logreg_gols

# Exemplo de jogo específico
jogo = {
    "HomeTeam": 'Arsenal',
    "AwayTeam": 'Man United',
}

# Fazer a sugestão de aposta com base no modelo treinado
sugestao_aposta = aposta_gols(jogo, modelo)

print(sugestao_aposta[0])

Sugestão de aposta: Mais de 2.5 gols com odd de 1.73


# Agora podemos comparar com o resultado na vida real:

![Resultado United x Arsenal](../imgs/unitedxarsenal2.png)

Assim como a iteração anterior, a acurácia não é perfeita, mas foi boa o suficiente para acertar o over 2.5 gols nesse jogo. Ainda assim, existem situações atípicas no futebol, como no jogo entre o Manchester United e o Chelsea, onde o United goleou o Chelsea por 4x1, mesmo os últimos 6 jogos entre os times tendo tido menos de 2.5 gols, como pode ser visto abaixo:

In [184]:
# Carregar o modelo treinado
modelo = svm_gols

# Exemplo de jogo específico
jogo = {
    "HomeTeam": 'Man United',
    "AwayTeam": 'Chelsea',
}

# Fazer a sugestão de aposta com base no modelo treinado
sugestao_aposta = aposta_gols(jogo, modelo)

print(sugestao_aposta[0])

Sugestão de aposta: Menos de 2.5 gols com odd de 2.38


# Agora podemos comparar com o resultado na vida real:

![Resultado United x Chelsea](../imgs/unitedxchelsea.png)

Dessa forma, é interessante medir a acurácia do modelo, para saber se ele é realmente bom. Para isso, vamos criar uma função que recebe os resultados reais e os resultados previstos, e retorna a acurácia do modelo, como pode ser visto abaixo:

In [174]:
def calcular_acuracia_gols(modelo):
    # Criar uma lista com os 20 times da Premier League 2021/2022
    times = lastseason['HomeTeam'].unique()

    # Criar uma acuracia total para todos os times
    acuracia_total = 0

    # Criar um for que percorre todos os times e faz a sugestão de aposta
    for time in times:
        # Criar um dataframe com os jogos do time
        jogos_time = lastseason[(lastseason['HomeTeam'] == time) | (lastseason['AwayTeam'] == time)].copy()
        jogos_time = jogos_time.drop(['AS', 'AST', 'HS', 'HST'], axis=1)

        # Criar um for que percorre todos os jogos do time e faz a sugestão de aposta
        for index, jogo in jogos_time.iterrows():
            # Obter as informações do jogo do lastseason
            home_team = jogo['HomeTeam']
            away_team = jogo['AwayTeam']
            jogo_info = lastseason[(lastseason['HomeTeam'] == home_team) & (lastseason['AwayTeam'] == away_team)].iloc[0]

            # Fazer a sugestão de aposta
            sugestao_aposta = aposta_gols(jogo, modelo)

            # Criar uma coluna com a sugestão de aposta
            jogos_time.loc[index, 'SugestaoAposta'] = sugestao_aposta[1]

            # Calcular o total de gols do jogo
            total_gols = jogo_info['FTHG'] + jogo_info['FTAG']

            # Definir o resultado do jogo com base no total de gols
            if total_gols > 2.5:
                jogos_time.loc[index, 'Resultado'] = 1.0
            else:
                jogos_time.loc[index, 'Resultado'] = 0.0


        # Criar uma coluna com o resultado da aposta
        jogos_time.loc[:, 'ResultadoAposta'] = jogos_time['Resultado'].eq(jogos_time['SugestaoAposta'])

        # Calcular a acurácia do modelo
        acuracia = jogos_time['ResultadoAposta'].mean()

        # Somar a acurácia do time na acurácia total
        acuracia_total += acuracia

    # Retornar a acurácia do modelo
    return acuracia_total / len(times)


Vamos testar a acurácia do Random Forest, que é uma evolução do Decision Tree, e o SVM, que é um modelo que utiliza vetores para classificar os dados. Para isso, vamos criar uma função que recebe o modelo, os dados de treino e os dados de teste, e retorna a acurácia do modelo, como pode ser visto abaixo:

In [175]:
print("A acurácia do modelo de regressão logística é de {}%".format(round(calcular_acuracia_gols(logreg_gols) * 100, 2)))
print("A acurácia do modelo de decision tree é de {}%".format(round(calcular_acuracia_gols(decision_tree_gols) * 100, 2)))
print("A acurácia do modelo de random forest é de {}%".format(round(calcular_acuracia_gols(random_forest_gols) * 100, 2)))
print("A acurácia do modelo support vector machine é de {}%".format(round(calcular_acuracia_gols(svm_gols) * 100, 2)))
print("A acurácia do modelo naive bayes é de {}%".format(round(calcular_acuracia_gols(naive_bayes_gols) * 100, 2)))

A acurácia do modelo de regressão logística é de 70.26%
A acurácia do modelo de decision tree é de 71.84%
A acurácia do modelo de random forest é de 71.84%
A acurácia do modelo support vector machine é de 68.95%
A acurácia do modelo naive bayes é de 56.84%


Como podemos ver, a acurácia desse modelo é melhor, prevendo no melhor dos casos com 75% de acurácia o over/under desses jogos. Com esses modelos, já fica interessante fazer uma aposta, possibilitando o lucro do usuário. Mesmo assim, *é importante deixar claro para o usuário que não recomendamos que ele realize bets esperando 100% de acerto ou garantia de lucro.*

O Futebol é um esporte imprevisível, e mesmo com um modelo que tem uma acurácia de 75%, ainda existem situações atípicas que podem acontecer, como lesões, suspensões, desfalques, etc. Por isso, é importante que o usuário tenha consciência de que o modelo não é perfeito, e que ele pode perder dinheiro com as apostas. O ideal seria juntar esse modelo com outros modelos, para que o usuário tenha uma maior segurança na hora de realizar a aposta.

# Playground para o usuário

Abaixo, vamos deixar um playground para o usuário, onde ele pode testar o modelo com os jogos que ele quiser, e ver se o modelo acerta ou não. Para isso, basta colocar o nome dos times que vão jogar, e o modelo que ele quer testar, como pode ser visto abaixo:

### Primeiro, vamos definir as funções:
se você não entende de programação, pode rodar o código sem se preocupar com isso

In [176]:
# Definir uma função em que o usuário insere o nome do time da casa, de fora e o modelo
def aposta_gols_usuario():
    # Obter o nome do time da casa
    home_team = input("Digite o nome do time da casa: ")

    # Obter o nome do time de fora
    away_team = input("Digite o nome do time de fora: ")

    # Obter o modelo
    modelo_resultado = input("Digite o nome do modelo para prever o resultado: ")

    # Definir o modelo de resultados buscando qual modelo o usuário escolheu
    if modelo_resultado == "logreg" or modelo_resultado == "regressão logística":
        modelo_resultado = logreg
    elif modelo_resultado == "decision_tree" or modelo_resultado == "decision tree":
        modelo_resultado = decision_tree
    elif modelo_resultado == "random_forest" or modelo_resultado == "random forest":
        modelo_resultado = random_forest
    elif modelo_resultado == "svm" or modelo_resultado == "support vector machine":
        modelo_resultado = svm
    elif modelo_resultado == "naive_bayes" or modelo_resultado == "naive bayes":
        modelo_resultado = naive_bayes

    # Obter o modelo de gols
    modelo_gols = input("Digite o nome do modelo para prever os gols: ")

    # Definir o modelo de gols buscando qual modelo o usuário escolheu
    if modelo_gols == "logreg_gols" or modelo_gols == "logreg" or modelo_gols == "regressão logística":
        modelo_gols = logreg_gols
    elif modelo_gols == "decision_tree_gols" or modelo_gols == "decision_tree" or modelo_gols == "decision tree":
        modelo_gols = decision_tree_gols
    elif modelo_gols == "random_forest_gols" or modelo_gols == "random_forest" or modelo_gols == "random forest":
        modelo_gols = random_forest_gols
    elif modelo_gols == "svm_gols" or modelo_gols == "svm" or modelo_gols == "support vector machine":
        modelo_gols = svm_gols
    elif modelo_gols == "naive_bayes_gols" or modelo_gols == "naive_bayes" or modelo_gols == "naive bayes":
        modelo_gols = naive_bayes_gols

    # Criar o dicionário com o jogo
    jogo = {
        "HomeTeam": home_team,
        "AwayTeam": away_team,
    }

    # Fazer a sugestão de aposta com base no modelo treinado
    resultado = aposta_resultado(jogo, modelo_resultado)

    # Fazer a sugestão de aposta com base no modelo treinado
    gols = aposta_gols(jogo, modelo_gols)

    print(resultado[0])
    print(gols[0])

    return jogo, modelo_resultado, modelo_gols, resultado[1], gols[1]

In [177]:
db = pathlib.Path.cwd() / "../data/seasons/2022-2023.csv"
lastseason = pd.read_csv(db)

# Criar uma função que retorna o resultado da aposta
def resultado_aposta(aposta):
    # Obter o resultado do jogo
    jogo = lastseason[(lastseason['HomeTeam'] == aposta[0]['HomeTeam']) & (lastseason['AwayTeam'] == aposta[0]['AwayTeam'])]

    jogo_info = lastseason[(lastseason['HomeTeam'] == aposta[0]['HomeTeam']) & (lastseason['AwayTeam'] == aposta[0]['AwayTeam'])].iloc[0]
    

    # Calcular o total de gols do jogo
    total_gols = jogo_info['FTHG'] + jogo_info['FTAG']

    # Definir o resultado do jogo com base no total de gols
    if total_gols > 2.5 and aposta[4] == 1.0:
        gols = "Acertamos que o jogo teve mais de 2.5 gols"
    elif total_gols < 2.5 and aposta[4] == 0.0:
        gols = "Acertamos que o jogo teve menos de 2.5 gols"
    else:
        gols = "Erramos o total de gols"
    
    # Definir se a aposta acertou ou não
    if aposta[3] == jogo_info['FTR']:
        resultado = "Acertamos o resultado!"
    else:
        resultado = "Erramos o resultado"
    

    # Criar uma lista com o resultado da aposta
    resultado_aposta = [resultado, gols]

    return resultado_aposta

## Agora vamos ver uma lista com os times dessa temporada!
O ideal é escrever o nome do time exatamente como está escrito na lista abaixo, para que o modelo funcione corretamente

In [178]:
# Criar uma lista com os 20 times da Premier League 2022/2023
times = lastseason['HomeTeam'].unique()

# Listar os times para o usuário
print("Lista de times da Premier League 2022/2023")
print(times)

Lista de times da Premier League 2022/2023
['Crystal Palace' 'Fulham' 'Bournemouth' 'Leeds' 'Newcastle' 'Tottenham'
 'Everton' 'Leicester' 'Man United' 'West Ham' 'Aston Villa' 'Arsenal'
 'Brighton' 'Man City' 'Southampton' 'Wolves' 'Brentford' "Nott'm Forest"
 'Chelsea' 'Liverpool']


## Lembrando

Para o modelo de previsão de resultados:

- A acurácia do modelo de regressão logística é de 61.84%
- A acurácia do modelo de decision tree é de 62.11%
- A acurácia do modelo de random forest é de 62.37%
- A acurácia do modelo support vector machine é de 61.32%
- A acurácia do modelo naive bayes é de 47.11%


Para o modelo de previsão de over/under 2.5 gols:

- A acurácia do modelo de regressão logística é de 70.26%
- A acurácia do modelo de decision tree é de 71.84%
- A acurácia do modelo de random forest é de 71.84%
- A acurácia do modelo support vector machine é de 68.95%
- A acurácia do modelo naive bayes é de 56.84%

Agora, você pode selecionar o time que você quiser e seu adversário, vendo a recomendação de bet para o jogo, se vai ter mais ou menos de 2.5 gols e as odds de cada opção, como pode ser visto abaixo:

In [180]:
sua_aposta = aposta_gols_usuario()

Sugestão de aposta: Vitória do time visitante com odd de 1.85
Sugestão de aposta: Mais de 2.5 gols com odd de 1.73


# Checando os resultados

Legal, né? Agora vamos checar se o modelo acertou os jogos que ele previu, como pode ser visto abaixo:

In [181]:
print(resultado_aposta(sua_aposta))

[    Div        Date   Time    HomeTeam  AwayTeam  FTHG  FTAG FTR  HTHG  HTAG  \
178  E0  14/01/2023  12:30  Man United  Man City     2     1   H     0     0   

     ... AvgC<2.5 AHCh  B365CAHH  B365CAHA  PCAHH  PCAHA  MaxCAHH  MaxCAHA  \
178  ...     2.22  0.5      2.06      1.87   2.02   1.91     2.06     1.95   

     AvgCAHH  AvgCAHA  
178      2.0     1.89  

[1 rows x 106 columns], 'Erramos o resultado', 'Acertamos que o jogo teve mais de 2.5 gols']


# Conclusão

Por fim, é relevante ressaltar que foi conquistado uma acurácia de 61.84% no modelo de previsão de resultados, e uma acurácia de 70.26% no modelo de previsão de over/under 2.5 gols. Esses resultados são bons, mas não são perfeitos, e por isso, é muito interessante como por mais que o futebol dependa muito de táticas e estátisticas, a imprevisibilidade ainda é um fator muito importante no esporte, tirando obviamente os casos de manipulação de resultados.

Possíveis outras justificativas para a diminuição da acurária dos modelos envolve a capacidade de desenvolvimento dos times, além da importância financeira. Quando pegamos dados individuais dos clubes, o newcastle é um dos que mais tem erro na previsão, já que até ano passado o time sofria com donos que não investiam no time, mas recentemente, o time foi comprado por um bilionário e o crescimento do time foi notável.

Além disso, possíveis iterações de apostas podem ser feitas, como por exemplo, apostar no resultado do primeiro tempo, ou até mesmo apostar no resultado do primeiro tempo e do segundo tempo. Outra iteração interessante seria apostar no resultado do jogo, mas com um handicap, que é uma vantagem que um time recebe antes do jogo começar, como por exemplo, o time A começa o jogo com 1 gol de vantagem, e o time B começa o jogo com 1 gol de desvantagem. Dessa forma, o time A só ganha a aposta se ganhar o jogo por 2 gols de diferença, e o time B só ganha a aposta se ganhar o jogo ou empatar. Além disso, existe a possibilidade de apostar em escanteios, cartões, e até mesmo em quem vai fazer o primeiro gol do jogo, mas o dataset não possui esses dados, e por isso, não foi possível fazer essas iterações.